### 使用VGG16作为预模型，加入类别权重以后，新定义的VGG16

In [ ]:
import tensorflow as tf
from tensorflow.keras import layers, models, optimizers
from tensorflow.keras.preprocessing.image import ImageDataGenerator

# 定义 VGG16 模型
class VGG16(tf.keras.Model):
    def __init__(self, num_classes=2):
        super(VGG16, self).__init__()
        self.features = models.Sequential([
            layers.Conv2D(64, (3, 3), padding='same', activation='relu', input_shape=(224, 224, 3)),
            layers.MaxPooling2D((2, 2), strides=(2, 2)),
            layers.Conv2D(128, (3, 3), padding='same', activation='relu'),
            layers.MaxPooling2D((2, 2), strides=(2, 2)),
            layers.Conv2D(256, (3, 3), padding='same', activation='relu'),
            layers.Conv2D(256, (3, 3), padding='same', activation='relu'),
            layers.MaxPooling2D((2, 2), strides=(2, 2)),
            layers.Conv2D(512, (3, 3), padding='same', activation='relu'),
            layers.Conv2D(512, (3, 3), padding='same', activation='relu'),
            layers.MaxPooling2D((2, 2), strides=(2, 2)),
            layers.Conv2D(512, (3, 3), padding='same', activation='relu'),
            layers.Conv2D(512, (3, 3), padding='same', activation='relu'),
            layers.MaxPooling2D((2, 2), strides=(2, 2)),
        ])
        self.classifier = models.Sequential([
            layers.Flatten(),
            layers.Dense(4096, activation='relu'),
            layers.Dropout(0.5),
            layers.Dense(4096, activation='relu'),
            layers.Dropout(0.5),
            layers.Dense(num_classes, activation='softmax'),
        ])

    def call(self, x):
        x = self.features(x)
        x = self.classifier(x)
        return x

# 加载预训练权重
model = VGG16(num_classes=2)
model.build(input_shape=(None, 224, 224, 3))
model.load_weights('Adogandcat_epoch_20.h5')  # 假设之前训练好的权重
# 冻结部分卷积层，仅微调分类器
for layer in model.features.layers[:-4]:
    layer.trainable = False

# 数据增强（训练集 + 验证集划分）
train_datagen = ImageDataGenerator(
    rescale=1.0 / 255,
    shear_range=0.2,
    zoom_range=0.2,
    rotation_range=20,
    horizontal_flip=True,
    validation_split=0.2  # 20% 数据作为验证集
)

# 训练数据生成器
train_gen = train_datagen.flow_from_directory(
    directory='data2/猫狗大战/train',  # 包含 cat 和 dog 两类
    target_size=(224, 224),
    batch_size=32,
    class_mode='binary',
    subset='training'  # 加载训练子集
)

# 验证数据生成器
val_gen = train_datagen.flow_from_directory(
    directory='data2/猫狗大战/train',  # 包含 cat 和 dog 两类
    target_size=(224, 224),
    batch_size=32,
    class_mode='binary',
    subset='validation'  # 加载验证子集
)

# 类别权重补偿（假设猫为 0，狗为 1）
class_weights = {
    0: 4.5,  # 猫的权重，假设在训练集中较少
    1: 1.0   # 狗的权重，假设在训练集中较多
}

# 编译模型
model.compile(
    optimizer=optimizers.Adam(learning_rate=0.0001),
    loss='sparse_categorical_crossentropy',
    metrics=['accuracy']
)

# 训练模型
history = model.fit(
    train_gen,
    steps_per_epoch=train_gen.samples // train_gen.batch_size,
    validation_data=val_gen,
    validation_steps=val_gen.samples // val_gen.batch_size,
    epochs=20,
    class_weight=class_weights  # 应用类别权重
)

model.save_weights('vgg16_dog_cat_classifier_with_split')

2024-11-22 16:56:12.231607: I tensorflow/core/platform/cpu_feature_guard.cc:142] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-11-22 16:56:12.666226: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1510] Created device /job:localhost/replica:0/task:0/device:GPU:0 with 38404 MB memory:  -> device: 0, name: NVIDIA A100-SXM4-40GB, pci bus id: 0000:13:00.0, compute capability: 8.0


OSError: Unable to open file (unable to open file: name = 'Adogandcat_epoch_20.h5', errno = 2, error message = 'No such file or directory', flags = 0, o_flags = 0)

In [24]:
model.summary()

Model: "vg_g16_14"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
sequential_30 (Sequential)   (None, 7, 7, 512)         9220480   
_________________________________________________________________
sequential_31 (Sequential)   (None, 2)                 119554050 
Total params: 128,774,530
Trainable params: 128,774,530
Non-trainable params: 0
_________________________________________________________________


### 预测代码

In [6]:
# 预测代码
import os
import pandas as pd
import numpy as np
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.models import load_model

# 加载模型
model=VGG16(num_classes=2)
model.build(input_shape=(None, 224, 224, 3))
model.load_weights('vgg16_dog_cat_classifier_with_split.h5')
print("模型加载完成！")

# 测试图片所在的文件夹路径
test_dir = 'data/test'  # 替换为你的测试数据目录
output_csv = 'prediction_results.csv'  # 输出预测结果的文件

# 测试数据生成器
test_datagen = ImageDataGenerator(rescale=1.0 / 255)
test_gen = test_datagen.flow_from_directory(
    directory=test_dir,  # 测试数据的文件夹路径
    target_size=(224, 224),
    batch_size=1,  # 每次预测一张图片
    class_mode=None,  # 测试数据没有标签
    shuffle=False  # 确保顺序一致
)

# 获取测试图片的文件名
filenames = test_gen.filenames

# 开始预测
print("开始预测...")
predictions = model.predict(test_gen, steps=len(test_gen), verbose=1)

# 将概率值转换为类别标签（0 表示猫，1 表示狗）
predicted_labels = ['cat' if pred[0] > pred[1] else 'dog' for pred in predictions]

# 保存预测结果到 CSV 文件
results = pd.DataFrame({
    'id': [os.path.basename(name) for name in filenames],
    'label': predicted_labels
})
results.to_csv(output_csv, index=False)

print(f"预测结果已保存到 {output_csv}")

ValueError: Cannot assign to variable conv2d_24/kernel:0 due to variable shape (3, 3, 3, 64) and value shape (512, 512, 3, 3) are incompatible

In [ ]:
import os
import pandas as pd
import numpy as np
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.utils import load_img, img_to_array

# 测试图片所在的文件夹路径
test_dir = 'data2/猫狗大战/test'
output_csv = 'prediction_results2.csv'

# 加载训练好的模型
model = VGG16(num_classes=2)
model.build(input_shape=(None, 224, 224, 3))
model.load_weights('VGG_weights/Adogandcat_epoch_18.h5')
print("模型权重加载完成！")

# 收集测试文件路径
file_paths = [os.path.join(test_dir, f) for f in os.listdir(test_dir) if f.endswith('.jpg')]

# 预测结果存储
predicted_labels = []
file_names = []

print("开始预测...")
for file_path in file_paths:
    # 加载并预处理图片
    img = load_img(file_path, target_size=(224, 224))
    img_array = img_to_array(img) / 255.0  # 归一化
    img_array = np.expand_dims(img_array, axis=0)  # 增加批次维度

    # 预测
    prediction = model.predict(img_array)
    label = 'cat' if np.argmax(prediction) == 0 else 'dog'
    predicted_labels.append(label)
    file_names.append(os.path.basename(file_path))

# 保存结果到 CSV 文件
results = pd.DataFrame({
    'id': file_names,
    'label': predicted_labels
})
results.to_csv(output_csv, index=False)

print(f"预测结果已保存到 {output_csv}")

import pandas as pd

# 读取 CSV 文件
csv_file = "prediction_results2.csv"
df = pd.read_csv(csv_file)

# 对 id 列进行排序（假设文件名的数字部分用于排序）
df['id_numeric'] = df['id'].str.extract(r'(\d+)').astype(int)  # 提取数字并转换为整数
df_sorted = df.sort_values(by='id_numeric').drop(columns=['id_numeric'])  # 按提取的数字排序

# 保存排序后的结果到新的 CSV 文件
sorted_csv_file = "sorted_prediction_results2.csv"
df_sorted.to_csv(sorted_csv_file, index=False)

print(f"排序后的结果已保存到 {sorted_csv_file}")



模型权重加载完成！
开始预测...
预测结果已保存到 prediction_results2.csv


In [28]:
import pandas as pd

# 读取 CSV 文件
csv_file = "prediction_results2.csv"
df = pd.read_csv(csv_file)

# 对 id 列进行排序（假设文件名的数字部分用于排序）
df['id_numeric'] = df['id'].str.extract(r'(\d+)').astype(int)  # 提取数字并转换为整数
df_sorted = df.sort_values(by='id_numeric').drop(columns=['id_numeric'])  # 按提取的数字排序

# 保存排序后的结果到新的 CSV 文件
sorted_csv_file = "sorted_prediction_results2.csv"
df_sorted.to_csv(sorted_csv_file, index=False)

print(f"排序后的结果已保存到 {sorted_csv_file}")


排序后的结果已保存到 sorted_prediction_results2.csv


### 使用VGG16定义的残差网络，但是这里效果很差

In [8]:
import tensorflow as tf
from tensorflow.keras import layers, models, optimizers
from tensorflow.keras.preprocessing.image import ImageDataGenerator

# 定义残差块
class ResidualBlock(tf.keras.layers.Layer):
    def __init__(self, filters, kernel_size=3, strides=1):
        super(ResidualBlock, self).__init__()
        self.conv1 = layers.Conv2D(filters, kernel_size, strides=strides, padding='same', activation='relu')
        self.conv2 = layers.Conv2D(filters, kernel_size, strides=1, padding='same', activation=None)
        self.shortcut = layers.Conv2D(filters, 1, strides=strides, padding='same', activation=None)  # 跳跃连接
        self.add = layers.Add()
        self.activation = layers.ReLU()

    def call(self, inputs):
        x = self.conv1(inputs)
        x = self.conv2(x)
        shortcut = self.shortcut(inputs)
        x = self.add([x, shortcut])  # 残差连接
        x = self.activation(x)
        return x

# 定义改进的 ResNet 网络
class VGGResNet(tf.keras.Model):
    def __init__(self, num_classes=2):
        super(VGGResNet, self).__init__()
        # 加载预训练 VGG16 基础模型
        self.vgg_base = tf.keras.applications.VGG16(include_top=False, weights=None, input_shape=(224, 224, 3))
        self.vgg_base.trainable = True  # 如果需要微调 VGG，则设置为 True

        # 添加残差模块
        self.res_block1 = ResidualBlock(filters=256)
        self.res_block2 = ResidualBlock(filters=512)

        # 分类头部
        self.global_pool = layers.GlobalAveragePooling2D()
        self.fc1 = layers.Dense(256, activation='relu')
        self.dropout = layers.Dropout(0.5)
        self.fc2 = layers.Dense(num_classes, activation='softmax')

    def call(self, inputs):
        x = self.vgg_base(inputs)
        x = self.res_block1(x)
        x = self.res_block2(x)
        x = self.global_pool(x)
        x = self.fc1(x)
        x = self.dropout(x)
        x = self.fc2(x)
        return x

# 加载训练好的 VGG 权重
weights_path = 'VGG_weights/Adogandcat_epoch_18.h5'  # 替换为您的权重文件路径
model = VGGResNet(num_classes=2)
model.build(input_shape=(None, 224, 224, 3))
model.vgg_base.load_weights(weights_path, by_name=True)
print("成功加载 VGG 预训练权重！")

# 数据增强
train_datagen = tf.keras.preprocessing.image.ImageDataGenerator(
    rescale=1.0 / 255,
    shear_range=0.2,
    zoom_range=0.2,
    rotation_range=20,
    horizontal_flip=True,
    validation_split=0.2  # 保留验证集
)

# 加载训练和验证数据
train_gen = train_datagen.flow_from_directory(
    directory='data2/猫狗大战/train',
    target_size=(224, 224),
    batch_size=32,
    class_mode='binary',
    subset='training'
)

val_gen = train_datagen.flow_from_directory(
    directory='data2/猫狗大战/train',
    target_size=(224, 224),
    batch_size=32,
    class_mode='binary',
    subset='validation'
)

# 定义类别权重（假设猫 1044 张，狗 4700 张）
class_weights = {
    0: 1.02,  # 猫的权重较高
    1: 1.0           # 狗的权重较低
}

# 编译模型
model.compile(
    optimizer=optimizers.Adam(learning_rate=0.0001),
    loss='sparse_categorical_crossentropy',
    metrics=['accuracy']
)

# 训练模型
history = model.fit(
    train_gen,
    validation_data=val_gen,
    epochs=10,
    class_weight=class_weights  # 应用类别权重
)



成功加载 VGG 预训练权重！
Found 16000 images belonging to 2 classes.
Found 4000 images belonging to 2 classes.
Epoch 1/10
500/500 [==============================] - 201s 400ms/step - loss: 0.7001 - accuracy: 0.4986 - val_loss: 0.6932 - val_accuracy: 0.5000
Epoch 2/10
  8/500 [..............................] - ETA: 1:48 - loss: 0.6997 - accuracy: 0.5234

KeyboardInterrupt: 

In [ ]:
# 保存模型
model.save('vgg_resnet_dog_cat_balanced.h5')
print("模型已保存！")

# 可视化训练结果
import matplotlib.pyplot as plt

plt.figure(figsize=(12, 4))

# 绘制训练和验证准确率
plt.subplot(1, 2, 1)
plt.plot(history.history['accuracy'], label='Train Accuracy')
plt.plot(history.history['val_accuracy'], label='Validation Accuracy')
plt.title('Accuracy')
plt.legend()

# 绘制训练和验证损失
plt.subplot(1, 2, 2)
plt.plot(history.history['loss'], label='Train Loss')
plt.plot(history.history['val_loss'], label='Validation Loss')
plt.title('Loss')
plt.legend()

plt.show()
